In [16]:
import pandas as pd
import os
import re

# Define paths
RAW_DATA_DIR = "data/raw/"
CLEANED_DATA_DIR = "data/cleaned/"
os.makedirs(CLEANED_DATA_DIR, exist_ok=True)

# File paths
RAW_FILE_PATH = os.path.join(RAW_DATA_DIR, "srd_patient_exit_interviews_raw.xlsx")
RENAMED_FILE_PATH = os.path.join(CLEANED_DATA_DIR, "srd_patient_exit_interviews_renamed.xlsx")

# Load the raw Excel file (main sheet)
try:
    excel_data = pd.read_excel(RAW_FILE_PATH, sheet_name=None)
    main_sheet_name = list(excel_data.keys())[0]  # Assume first sheet is main
    df = excel_data[main_sheet_name]
    print(f"📂 Loaded raw data from: {RAW_FILE_PATH}")
    print(f"Original headers: {df.columns.tolist()}")
except FileNotFoundError:
    print(f"❌ Error: File not found at {RAW_FILE_PATH}. Please ensure the file exists.")
    exit(1)

# Function to clean headers
def clean_column_name(col):
    col = re.sub(r'[^a-zA-Z0-9\s]', '', col)  # Keep alphanumeric and spaces
    col = col.replace(' ', '_').lower().strip()  # Replace spaces with underscores, lowercase
    return col

# Apply cleaning to headers
df.columns = [clean_column_name(col) for col in df.columns]
print("\nCleaned headers:")
print(df.columns.tolist())

# Define header renaming mapping (based on cleaned headers)
header_mapping = {
    'start': 'start_time',
    'end': 'end_time',
    'deviceid': 'device_id',
    'governorate': 'region',
    'district': 'district',
    'sub_district': 'sub_district',
    'community': 'community',
    'grant': 'grant',
    'facility': 'health_facility',
    'other_health_facility': 'other_facility',
    'hello_my_name_is_and_i_work_for_name_of_partner_organization_thank_you_for_making_the_time_to_speak_to_us_we_are_conducting_srh_services_exit_interview_kindly_note_that_your_answers_will_be_treated_confidentially_and_will_be_used_to_improve_the_quality_of_our_services_to_the_beneficiaries_your_identity_will_be_kept_confidential_this_interview_will_take_approximately_20__30_minuteswe_want_to_assure_you_that_your_participation_in_this_study_is_voluntary_you_have_the_right_to_withdraw_from_the_interview_anytime_during_the_interview_you_have_the_right_to_refuse_to_answer_any_question_would_you_like_to_proceed_with_the_interview': 'intro_consent',
    'the_date_of_the_questionnaire': 'survey_date',
    'observer_name': 'observer',
    'is_the_patient_child_or_adult_child_under_18_adult_18_or_above': 'patient_age_group',
    'sex_of_the_respondent': 'respondent_sex',
    'how_old_is_the_respondent_in_years': 'respondent_age',
    'how_old_is_the_patient_in_years': 'patient_age',
    'is_this_your_first_visit': 'first_visit',
    'what_is_your_marital_status': 'marital_status',
    'how_many_children_do_you_have': 'children_count',
    'how_old_is_your_oldest_child': 'oldest_child_age',
    'do_you_have_difficulty_seeing_even_if_wearing_glasses': 'vision_difficulty',
    'do_you_have_difficulty_hearing_even_if_using_a_hearing_aid': 'hearing_difficulty',
    'do_you_have_difficulty_walking_or_climbing_steps': 'mobility_difficulty',
    'using_your_usual_customary_language_do_you_have_difficulty_communicating_for_example_understanding_or_being_understood': 'communication_difficulty',
    'do_you_have_difficulty_remembering_or_concentrating': 'cognitive_difficulty',
    'was_it_easy_for_you_to_reach_the_site_of_the_facility_today': 'facility_access',
    'what_does_make_the_access_easy': 'access_factors',
    'if_not_easy_please_specify_why': 'access_issues',
    'if_other_answer_please_specify': 'other_access_issues',
    'how_did_you_find_out_about_the_services_here': 'service_discovery',
    'if_other_please_specify': 'other_discovery',
    'what_made_you_select_this_one': 'selection_reason',
    'if_other_answer_please_specify': 'other_selection',
    'in_general_what_was_the_quality_of_the_services_you_received_in_this_facility': 'service_quality',
    'if_it_was_bad_or_very_bad_according_to_you_please_explain_why': 'bad_service_reason',
    'which_services_did_you_receive_today': 'services_received',
    'which_services_did_you_receive_todaygeneral_clinic': 'general_clinic',
    'which_services_did_you_receive_todaygyneocology': 'gynecology',
    'which_services_did_you_receive_todaypediatric': 'pediatrics',
    'which_services_did_you_receive_todaynutrition': 'nutrition',
    'which_services_did_you_receive_todayother': 'other_services',
    'which_services_did_you_receive_today5': 'birth_services',
    'birth_services': 'birth_services_specify',
    'spicify_others': 'other_services_specify',
    'were_you_charged_for_any_of_the_services_you_received': 'service_charge',
    'if_yes_which_ones': 'charged_services',
    'if_yes_which_onesgeneral_clinic': 'general_clinic_charge',
    'if_yes_which_onesgyneocology': 'gynecology_charge',
    'if_yes_which_onespediatric': 'pediatrics_charge',
    'if_yes_which_onesnutrition': 'nutrition_charge',
    'if_yes_which_onesother': 'other_services_charge',
    'if_yes_which_ones5': 'birth_services_charge',
    'spicify_others1': 'other_charge_specify',
    'are_there_any_services_you_needed_that_were_not_available_in_this_center': 'services_unavailable',
    'if_yes_which_ones1': 'unavailable_services',
    'can_you_mention_any_change_in_terms_of_your_health_or_the_health_of_your_children_positive_or_negative_as_a_result_of_the_services_provided_by_this_facility': 'health_impact',
    'are_there_any_close_health_facility_that_provide_these_services': 'nearby_facilities',
    'can_you_access_those_facilities': 'nearby_access',
    'if_no_please_explain_why': 'access_barriers',
    'if_yes_please_eaxplain_then_why_you_chose_to_reasieve_the_services_from_this_facility': 'facility_choice',
    'were_you_prescribed_any_medicines': 'prescribed_meds',
    'if_yes_did_you_received_medicine_prescribed': 'meds_received',
    'did_you_feel_confident_that_you_received_a_thorough_physical_exam': 'exam_confidence',
    'did_the_provider_explain_how_you_should_use_the_drugs_you_received': 'meds_explained',
    'did_the_provider_explain_when_to_return': 'follow_up_explained',
    'did_you_receive_the_service_you_came_to_this_facility_for': 'service_received',
    'mention_how_long_it_took_to_get_the_service': 'service_duration',
    'waiting_time': 'waiting_time',
    'time_with_the_health_provider': 'provider_time',
    'privacy_during_examination': 'exam_privacy',
    'staff_attitude': 'staff_attitude',
    'opening_hours_of_the_facility': 'opening_hours',
    'quality_of_the_advice_and_information': 'advice_quality',
    'procedure_or_treatment': 'treatment_quality',
    'how_would_you_describe_your_overall_satisfaction_with_the_overall_service_you_received': 'satisfaction',
    'if_not_satisfied_or_somewhat_satisfied_above_please_explain_why': 'dissatisfaction_reason',
    'overall_cleanliness_of_the_health_facility': 'facility_cleanliness',
    'clinic_conditions_renovation_equipment_supplies': 'clinic_conditions',
    'services_rendered_and_hours_of_service_are_clearly_displayed_on_a_board_at_the_facility': 'services_displayed',
    'the_private_places_to_talk_with_clinic_staff': 'private_spaces',
    'the_ease_of_moving_around_the_clinic': 'clinic_mobility',
    'comfortable_places_to_sit_while_waiting_to_be_assisted_by_facility_staff': 'waiting_comfort',
    'did_you_feel_safe_at_all_times_travelling_to_receive_the_assistanceservice': 'travel_safety',
    'if_no_what_could_have_been_done_by_the_organization_to_make_you_feel_safer': 'safety_suggestions',
    'did_you_feel_that_the_agencyngoimplementing_partnercontractor_staff_treated_you_with_respect_during_the_the_providing_the_serviceaid': 'staff_respect',
    'if_no_would_you_mind_telling_us_when_or_where_would_you_mind_telling_us_why': 'disrespect_reason',
    'are_you_satisfied_with_the_assistanceservice_provided': 'service_satisfaction',
    'if_no_would_you_mind_telling_us_why_you_are_not_satisfied': 'service_dissatisfaction',
    'do_you_know_of_people_needing_assistanceservices_who_were_excluded_from_the_assistanceservice_provided': 'excluded_people',
    'if_yes_who_was_mainly_excluded': 'excluded_groups',
    'if_you_had_a_suggestion_for_or_a_problem_with_the_assistanceservice_do_you_think_you_could_channel_the_suggestion_or_lodge_a_complaint': 'complaint_channel',
    'to_your_knowledge_have_suggestions_or_complaints_raised_been_responded_to_or_followed_up': 'complaint_response',
    'if_no_would_you_mind_telling_me_which_are_the_issues_what_happened': 'complaint_issues',
    'were_your_views_taken_into_account_by_the_organization_about_the_assistance_you_received': 'views_considered',
    'if_no_would_you_mind_telling_me_how_is_it_that_your_views_were_not_taken_into_account': 'views_ignored_reason',
    'did_you_feel_well_informed_about_the_assistanceservice_available': 'informed_services',
    'if_no_what_could_the_aidservice_provider_have_done_to_better_inform_you_about_the_assistance_services_available_to_you': 'information_gap',
    'were_your_views_taken_into_account_by_the_organization_about_the_assistance_you_received1': 'views_considered_2',
    'if_no_would_you_mind_telling_me_how_is_it_that_your_views_were_not_taken_into_account1': 'views_ignored_reason_2',
    'if_you_a_have_complaint_or_feedback_do_you_the_available_channels': 'feedback_channels',
    'if_you_a_have_complaint_or_feedback_do_you_the_available_channelscomplaint_box': 'complaint_box',
    'if_you_a_have_complaint_or_feedback_do_you_the_available_channelsinformation_desk': 'info_desk',
    'if_you_a_have_complaint_or_feedback_do_you_the_available_channelsngo_staff_membersvolunteers': 'ngo_staff',
    'if_you_a_have_complaint_or_feedback_do_you_the_available_channelsphone_line': 'phone_line',
    'if_you_a_have_complaint_or_feedback_do_you_the_available_channelswhatsappviber': 'whatsapp_viber',
    'if_you_a_have_complaint_or_feedback_do_you_the_available_channelssocial_media': 'social_media',
    'if_you_a_have_complaint_or_feedback_do_you_the_available_channelsthrough_the_local_council': 'local_council',
    'if_you_a_have_complaint_or_feedback_do_you_the_available_channelsother': 'other_feedback',
    'other_specify': 'other_feedback_specify',
    'are_you_satisfied_in_general_with_the_support_that_you_received': 'general_satisfaction',
    'if_no_why': 'general_dissatisfaction',
    'do_you_have_any_other_feedback_or_suggestions_to_improve_our_service_yesno': 'feedback_available',
    'if_yes_can_you_please_share_it_with_us': 'feedback_details',
    'id': 'response_id',
    'uuid': 'uuid',
    'submission_time': 'submission_time',
    'validation_status': 'validation_status',
    'notes': 'notes',
    'status': 'status',
    'submitted_by': 'submitted_by',
    'version': 'version',
    'tags': 'tags',
    'index': 'index'
}

# Apply renaming
df.rename(columns=header_mapping, inplace=True)

# Save renamed DataFrame
df.to_excel(RENAMED_FILE_PATH, index=False)
print(f"📁 Renamed data saved to: {RENAMED_FILE_PATH}")

# Verify the saved file
df_verified = pd.read_excel(RENAMED_FILE_PATH)
print("\nRenamed headers:")
print(df_verified.columns.tolist())
print("\nFirst 5 rows of renamed data:")
print(df_verified.head())

📂 Loaded raw data from: data/raw/srd_patient_exit_interviews_raw.xlsx
Original headers: ['start', 'end', 'deviceid', 'governorate', 'district', 'sub district', 'community', 'grant', 'Health Facility', 'Other Health Facility', 'Hello "My name is .........and I work for (name of partner organization). Thank you for making the time to speak to us, we are conducting SRH services exit interview. Kindly note that your answers will be treated confidentially, and will be used to improve the quality of our services to the beneficiaries. Your identity will be kept confidential. This interview will take approximately 20 – 30 minutes.We want to assure you that:- Your participation in this study is voluntary;- You have the right to withdraw from the interview anytime during the interview;- You have the right to refuse to answer any question;- Would you like to proceed with the interview?', 'The date of the questionnaire', 'Observer name', 'Is the patient Child or Adult? (Child= under 18, Adult= 18 

In [17]:
print("\nHeaders before renaming (should match header_mapping keys):")
unmatched = [col for col in df.columns if col not in header_mapping]
print(f"Unmatched headers: {unmatched}")


Headers before renaming (should match header_mapping keys):
Unmatched headers: ['start_time', 'end_time', 'device_id', 'region', 'health_facility', 'other_facility', 'intro_consent', 'survey_date', 'observer', 'patient_age_group', 'respondent_sex', 'respondent_age', 'patient_age', 'first_visit', 'marital_status', 'children_count', 'oldest_child_age', 'vision_difficulty', 'hearing_difficulty', 'mobility_difficulty', 'communication_difficulty', 'cognitive_difficulty', 'facility_access', 'access_factors', 'access_issues', 'other_selection', 'service_discovery', 'other_discovery', 'selection_reason', 'other_selection', 'service_quality', 'bad_service_reason', 'services_received', 'general_clinic', 'gynecology', 'pediatrics', 'nutrition', 'other_services', 'birthservices', 'spicifyothers', 'service_charge', 'charged_services', 'general_clinic_charge', 'gynecology_charge', 'pediatrics_charge', 'nutrition_charge', 'other_services_charge', 'birth_services_charge', 'spicifyothers1', 'services_

In [21]:
import pandas as pd

# Load the renamed data
df = pd.read_excel(os.path.join(CLEANED_DATA_DIR, "srd_patient_exit_interviews_renamed.xlsx"))
print(f"📂 Loaded renamed data from: {RENAMED_FILE_PATH}")

# Basic description
print("\nDataset Info:")
print(df.info())

# Summary statistics
print("\nSummary Statistics (numeric columns):")
print(df.describe())

# Missing values
print("\nMissing Values by Column:")
missing = df.isnull().sum()
print(missing[missing > 0])

# Unique values for categorical columns
categorical_cols = ['region', 'district', 'sub_district', 'community', 'facility', 'patient_age_group', 'respondent_sex', 'marital_status', 'service_quality', 'satisfaction', 'general_satisfaction']
print("\nUnique Values for Categorical Columns:")
for col in categorical_cols:
    if col in df.columns:
        print(f"{col}: {df[col].unique().tolist()}")

# Sample data
print("\nSample Data (first 5 rows):")
display(df.head())

📂 Loaded renamed data from: ../data/cleaned/srd_patient_exit_interviews_renamed.xlsx

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766 entries, 0 to 765
Columns: 124 entries, start_time to index
dtypes: float64(33), int64(9), object(82)
memory usage: 742.2+ KB
None

Summary Statistics (numeric columns):
       respondent_age  first_visit  children_count  oldest_child_age  \
count      733.000000   766.000000      367.000000        326.000000   
mean        32.251023     1.795039        3.765668         11.932515   
std         11.466142     0.403937        2.510006          9.899264   
min         15.000000     1.000000        0.000000          1.000000   
25%         24.000000     2.000000        2.000000          5.000000   
50%         30.000000     2.000000        3.000000          9.000000   
75%         38.000000     2.000000        6.000000         15.000000   
max         96.000000     2.000000       16.000000         75.000000   

       general_clinic  gyne

,start_time,end_time,device_id,region,district,sub_district,community,grant,health_facility,other_facility,...,response_id,uuid,submissiontime,validationstatus,notes,status,submittedby,version,tags,index
0,2024-11-17T13:09:47.980+03:00,2024-11-17T13:14:40.802+03:00,collect:gKDiCHawM8MKgbhd,Aleppo,A'zaz,Aghtrin,Akhtrein,NaN,Akhtarin Hospital,NaN,...,621538440,56ed76f0-83ae-4cb8-9a4b-391986cc529e,2024-11-17T11:25:13,NaN,NaN,submitted_via_web,NaN,vHs5iKYL9VJwWuUKrgPMMf,NaN,1
1,2024-11-17T13:14:50.112+03:00,2024-11-17T13:17:26.376+03:00,collect:gKDiCHawM8MKgbhd,Aleppo,A'zaz,Aghtrin,Akhtrein,NaN,Akhtarin Hospital,NaN,...,621538447,49a19a1c-ab27-40fc-8c2f-0954f0275c52,2024-11-17T11:25:15,NaN,NaN,submitted_via_web,NaN,vHs5iKYL9VJwWuUKrgPMMf,NaN,2
2,2024-11-17T13:17:40.724+03:00,2024-11-17T13:20:33.101+03:00,collect:gKDiCHawM8MKgbhd,Aleppo,A'zaz,Aghtrin,Akhtrein,NaN,Akhtarin Hospital,NaN,...,621538455,c6f3ca47-f08a-4393-a40b-647a217eddb8,2024-11-17T11:25:17,NaN,NaN,submitted_via_web,NaN,vHs5iKYL9VJwWuUKrgPMMf,NaN,3
3,2024-11-17T13:21:04.391+03:00,2024-11-17T13:23:44.535+03:00,collect:gKDiCHawM8MKgbhd,Aleppo,A'zaz,Aghtrin,Akhtrein,NaN,Akhtarin Hospital,NaN,...,621538458,b5f4bfab-1029-42df-b56d-381b375cad95,2024-11-17T11:25:18,NaN,NaN,submitted_via_web,NaN,vHs5iKYL9VJwWuUKrgPMMf,NaN,4
4,2024-11-17T13:24:32.793+03:00,2024-11-17T13:28:08.694+03:00,collect:gKDiCHawM8MKgbhd,Aleppo,A'zaz,Aghtrin,Akhtrein,NaN,Akhtarin Hospital,NaN,...,621538461,6c6b9432-878a-4a09-8ff0-1a0073b6bd37,2024-11-17T11:25:19,NaN,NaN,submitted_via_web,NaN,vHs5iKYL9VJwWuUKrgPMMf,NaN,5
